# Lab 5

<b>Tarnavskyi Maksym
<br>Trokhymovych Mykola
<br>Golovko Andriy</b>

In [29]:
import numpy as np
import time
import pandas as pd
from scipy.linalg import inv

#for beatiful visualization
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly 
import plotly.graph_objs as go
from plotly import tools
import plotly.offline as py

from numpy.linalg import solve

init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

In [2]:
eps = 10 ** -5
A = 2*np.array([[1., 0.001 / 2],
              [0.001 / 2, 4.]])
B = np.array([0., -1.])
C = 0

In [3]:
def func(x):
    return .5*A.dot(x).dot(x) + B.dot(x) + C


def norm(x):
    return sum([i ** 2 for i in x]) ** 0.5


def unit_vector(i, n):
    return np.array([0 if i != j else 1 for j in range(n)])


def derivative(f, x, h=eps):
    n = len(x)
    grad = [(f(x + unit_vector(i, n).dot(h)) - f(x - unit_vector(i, n).dot(h))) / (2. * h) for i in range(n)]
    return np.array(grad)

def second_derivative(f, x, h=eps):
    n = len(x)
    hesse = [np.array([(f(x + (unit_vector(i, n) + unit_vector(j, n)) * h)
                        - f(x + (unit_vector(i, n) - unit_vector(j, n)) * h)
                        - f(x - (unit_vector(i, n) - unit_vector(j, n)) * h)
                        + f(x - (unit_vector(i, n) + unit_vector(j, n)) * h))
                       / (4. * h ** 2) for j in range(n)]) for i in range(n)]
    return np.array(hesse)

In [5]:
def choose_step_fragmentation(f, x, h, beta=1, λ=0.5, eps=eps):
    alpha = beta
    while f(x + alpha * h) - f(x) > .5 * alpha * derivative(f, x, eps).dot(h):
        alpha *= λ
    return alpha


def choose_step_fastest(f, x, h, method):
    def f_a(a, f=f, x=x, h=h):
        return f(x + a * h)
    return minimize_one_dimension(f_a, method)

def choose_step(f, x, h, method):
    # if method == "fragmentation":
    #     return choose_step_fragmentation(f, x, h)
    if method == "fastest_golden_ratio":
        return choose_step_fastest(f, x, h, method="golden_ratio")
    elif method == "fastest_brute_force":
        return choose_step_fastest(f, x, h, method="brute_force")
    elif method == "fastest_half_divide_method":
        return choose_step_fastest(f, x, h, method="half_divide_method")
    elif method == "fastest_fibonachi":
        return choose_step_fastest(f, x, h, method="fibonachi")
    elif method == "fastest_parabol":
        return choose_step_fastest(f, x, h, method="parabol")
    elif method == "fastest_newtons_method":
        return choose_step_fastest(f, x, h, method="newtons_method")

### Select one dimension function

In [10]:
def minimize_one_dimension(f, method,b=1 / eps,a=-1 / eps, eps=eps):
    if method == "golden_ratio":
        return minimize_one_dimension_golden_ratio(f,b,a)
    elif method == "brute_force":
        return minimize_one_dimension_brute_force(f, b=1 , a=-1 , n=int(eps ** -.5))
    elif method == "half_divide_method":
        return minimize_one_dimension_half_divide_method(f, b,a)
    elif method == "fibonachi":
        return minimize_one_dimension_fibonachi(f,b,a)
    elif method == "parabol":
        return minimize_one_dimension_parabol(f, b=1 / eps, a=-1 / eps)
    elif method == "newtons_method":
        return minimize_one_dimension_newtons_method(f, b=1 / eps, a=-1 / eps)
    elif method == "cubic_interpolation":
        return cubic_interpolation(f, lambda x: f1(f, x,eps), a, b)

In [11]:
def b_a(x0,h):
    b = x0+ h
    a = x0 - h
    return b,a

In [12]:
def test(func,x0,h,method, verbose=True):
    b,a = b_a(x0,h)
    start_time = time.time()
    x_min, n_i = minimize_one_dimension(func, method, b, a)
    dif_time = time.time() - start_time
    if verbose:
        print('x_min:', x_min)
        print('num_iter: ',n_i)
        print('time: ',dif_time)
    return x_min, n_i, dif_time

### One dimension functions

## Newtons method

In [13]:
def minimize_one_dimension_newtons_method(f, b, a,e=eps):
    i = 0
    start_time = time.time()
    x0 = float(b-a)/2. + a
    while True:
        i += 1
        f_1 = f1(f,x0,e)
        if abs(f_1) < e:
            return x0, i
        x1 = x0 - (f(x0) / f_1)
        if abs(x1 - x0) < e:
            return x1, i 
        x0 = x1
        if i > 10**6:
            return np.nan, i
        
def f1(f,x,h):
    return float(f(x + h) - f(x-h))/(2.* h)

In [14]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time = test(fun1,1.5,1,'newtons_method')

x_min: 1.000000291960698
num_iter:  358542
time:  0.3660297393798828


## Golden ratio

In [15]:
def minimize_one_dimension_golden_ratio(f, b=1 / eps, a=-1 / eps, eps=eps):
    F = (1. + 5 ** 0.5) / 2
    i = 0
    while abs(b - a) > eps:
        i += 1
        x1 = b - (b - a) / F
        x2 = a + (b - a) / F
        if f(x1) >= f(x2):
            a = x1
        else:
            b = x2
    return (a + b) / 2. , i 

In [16]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time =test(fun1,1.5,1,'golden_ratio')

x_min: 0.9999988615768525
num_iter:  26
time:  0.0001270771026611328


## Brute force

In [17]:
def minimize_one_dimension_brute_force(f, b, a, n = int(1 / eps)):
    i = 0
    x_min = a + (b - a) / (n + 1)
    for i in range(2, n + 1):
        x = a + i * (b - a) / (n + 1)
        x_min = x if f(x) < f(x_min) else x_min
        i += 1
    return x_min, i

#minimize_one_dimension_brute_force(func, 3, 0)

In [18]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time =test(fun1,1,1,'brute_force')

x_min: 0.9936908517350158
num_iter:  317
time:  0.00093841552734375


# Fibonachi

In [19]:
def minimize_one_dimension_fibonachi(f, b=1 / eps, a=-1 / eps, eps=eps):
    vars = round((b-a)/eps)
    n = get_iteration(vars)+1
    x1 = a+(fibonachi(n-2)/fibonachi(n))*(b-a)
    x2 = a+(fibonachi(n-1)/fibonachi(n))*(b-a)
    k=0
    i = 0
    while(1):
        i += 1
        if(f(x1)>=f(x2)):
            a = x1
            x1 = x2 
            x2 = a+(fibonachi(n-k-2)/fibonachi(n-k-1))*(b-a)
        else:
            b = x2
            x2 = x1
            x1 = a+(fibonachi(n-k-3)/fibonachi(n-k-1))*(b-a) 
        k = k+1
        if(abs(b-a)<eps):
            break;
    return (a+b)/2 , i

In [20]:
def fibonachi(n):
    i = 1
    j = i
    c = 1
    while(c<n):
        k = i
        i = j
        j = k+i
        c = c+1
    return i 

def get_iteration(n):
    i = 1
    j = i
    c = 0
    while(i<=n):
        k = i
        i = j
        j = k+i
        c = c+1
    return c

In [21]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time = test(fun1,1.5,1,'fibonachi')

x_min: 1.000004719786288
num_iter:  26
time:  0.0006439685821533203


## Half divide method

In [22]:
def minimize_one_dimension_half_divide_method(f, b=1 / eps, a=-1 / eps, e=eps):
    i = 0
    while(1):
        i += 1
        a = float(a)
        b = float(b)
        x1 = float((a+b-e))/2
        x2 = float((a+b+e))/2
        y1 = f(x1)
        y2 = f(x2)
        if(y1>y2):
            a = x1
        else:
            b = x2
        if((b-a)<2*e):
            break;
    return((a+b)/2) , i

In [23]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time = test(fun1,1.5,1,'half_divide_method')

x_min: 0.9999986853218078
num_iter:  18
time:  3.743171691894531e-05


## Parabol

In [24]:
def minimize_one_dimension_parabol(f,b,a,e=eps,h=0.5):
    i = 0
    x0 = float(b - a)/2. + a
    x1 = x0 - h
    x2 = x0
    x3 = x0 + h
    y1 = f(x1)
    y2 = f(x2)
    y3 = f(x3)

    if y1-2*y2+y3 < 0:
        print("Bad x0")
        return np.nan

    while 1:
        z1 = x1 - x3
        z2 = x2 - x3
        if abs(z1) < e:
            z1 = e
        p = float(((y1 - y3)*z2 - (y2-y3)*z1))/(z1*z2*(z1-z2))
        q = float(((y1-y3)*(z2**2) - (y2-y3)*(z1**2)))/(z1*z2*(z2-z1))
        if abs(p) < e:
            p = e
        zm = -q/(2*p)
        x1 = x2
        x2 = x3
        y1 = y2
        y2 = y3
        x3 = x3 + zm
        y3 = f(x3)
        
        i += 1
        
        if i > 10**6:
            return np.nan, i
        
        if abs(zm) < e:
            answer = x3 + zm
            return answer, i

In [25]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time = test(fun1,1.5,1,'parabol')

x_min: 1.0
num_iter:  2
time:  2.2649765014648438e-05


## Cubic interpolation

In [26]:
def cubic_interpolation(f, grad, x1, x2, max_iter=100, eps=10e-8):
    if grad(x1) > 0:
        print('error 101: grad(x1) must be <= 0')
        return np.nan, 0
    if grad(x2) < 0:
        print('error 102: grad(x2) must be >= 0')
        return np.nan, 0
    
    x3 = np.inf
    for i in range(max_iter):
        x3_ = x3
        
        
        X = np.array([[x1**3, x1 ** 2, x1, 1],
                     [3 * x1 **2, 2 * x1, 1, 0],
                     [x2**3, x2 ** 2, x2, 1],
                     [3 * x2 **2, 2 * x2, 1, 0]])
        v = np.array([f(x1), grad(x1), f(x2), grad(x2)])
        a, b, c, d = tuple(solve(X, v))
        #print(a, b, c, d)
        
        if abs(a) > 10e-8:
            D = np.sqrt(b**2 - 3*a*c)
            x_1 = (-b - D) / 3 / a
            x_2 = (-b + D) / 3 / a
            #print('x_1, x_2 = ', x_1, x_2)

            if f(x_1) < f(x_2):
                x3 = x_1
            else:
                x3 = x_2
        elif abs(b) > 10e-8:
            x3 = - c / 2 /b
        else:
            print('error 102')

        
        if grad(x3) > 0:
            x2 = x3
        else:
            x1 = x3

        if i > 0:
            if abs(x3 - x3_) < eps:
                break
    
    return x3, i

In [30]:
fun1 = lambda x: (x-1)**2 + 2
x_min, n_i, dif_time = test(fun1,1.5,1,'cubic_interpolation')

x_min: 0.999999999851
num_iter:  1
time:  0.027924299240112305


## For n-dimension fuctions

In [31]:
def minimize_gradient(f, x0, step_method, eps=eps, output=0):
    i = 0
    x = np.copy(x0)
    h = -derivative(f, x, eps)
    alpha, j = choose_step(f, x, h, step_method)
    #print("x{0} = {1}\nα{0} = {2}\nf(x{0}) = {3}".format(i, x, alpha, f(x)))
    x1 = x + alpha * h
    while norm(x1 - x) > eps:
        i += 1
        x = np.copy(x1)
        h = -derivative(f, x)
        alpha = choose_step(f, x, h, step_method)
        #print("x{0} = {1}\nα{0} = {2}\nf(x{0}) = {3}".format(i, x, alpha, f(x))) if i <= output else None
        x1 = x + alpha * h
    #print("{0} iterations".format(i+1))
    return x1, i+1


In [32]:
def minimize_newton(f, x0, step_method, eps=eps, output=0):
    i = 0
    x = np.copy(x0)
    h = -inv(second_derivative(f, x, eps)).dot(derivative(f, x, eps))
    alpha, j = choose_step(f, x, h, step_method)
    print("x{0} = {1}\nα{0} = {2}\nf(x{0}) = {3}".format(i, x, alpha, f(x)))
    x1 = x + alpha * h
    while norm(x1 - x) > eps:
        i += 1
        x = np.copy(x1)
        h = -inv(second_derivative(f, x, eps)).dot(derivative(f, x, eps))
        alpha = choose_step(f, x, h, step_method)
        print("x{0} = {1}\nα{0} = {2}\nf(x{0}) = {3}".format(i, x, alpha, f(x))) if i <= output else None
        x1 = x + alpha * h
    return x1

### Visualization functions

In [33]:
def compare_bar(df):
    trace1 = go.Bar(x=df.method.values, y=df.num_iterations.values)
    trace2 = go.Bar(x=df.method.values, y=df.time_duration.values)

    fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Number iterations', 'Time duration'))

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 2)

    fig['layout']['xaxis1'].update(title='Method')
    fig['layout']['xaxis2'].update(title='Method')

    fig['layout']['yaxis1'].update(title='Number iterations')
    fig['layout']['yaxis2'].update(title='Time duration')

    fig['layout'].update(showlegend=False, height=600, width=900, title='Comparing methods')
    py.iplot(fig, filename='simple-subplot')

In [34]:
def show_iters(df_iter):
    data = []
    for method in methods:
        trace = go.Scatter(
            x = steps,
            y = df_iter.loc[:,method].values,
            mode = 'lines+markers',
            name = method
        )

        data.append(trace)

    layout = go.Layout(
        showlegend=True,
        title='Number of operations per length of interval',
        xaxis=dict(title='Length of interval'),
        yaxis=dict(title='Number of iterations')
    )
    fig = go.Figure(data=data, layout=layout)

    py.iplot(fig, filename='line-mode')

In [35]:
def show_time(df_time):
    data = []
    for method in methods:
        trace = go.Scatter(
            x = steps,
            y = df_time.loc[:,method].values,
            mode = 'lines+markers',
            name = method
        )

        data.append(trace)

    layout = go.Layout(
        showlegend=True,
        title='Time of operations per length of interval',
        xaxis=dict(title='Length of interval'),
        yaxis=dict(title='Time')
    )
    fig = go.Figure(data=data, layout=layout)

    py.iplot(fig, filename='line-mode')

In [36]:
def fun_2d(func,x0,h,N=100):
    b,a = b_a(x0,h)
    X = np.linspace(a, b, N)
    Y = np.array(list(map(lambda x: fun1(x),X)))

    trace1 = go.Scatter(x=X, y=Y, name='function')

    data=go.Data([trace1])

    layout=go.Layout(title="Target function", xaxis={'title':'x'}, yaxis={'title':'f(x)'})
    figure=go.Figure(data=data,layout=layout)
    py.iplot(figure, filename='pyguide_1')

In [37]:
def show_func(func,b=10,a=-10,N=100):
    
    X = np.linspace(a, b, N)
    Y = np.linspace(a, b, N)
    Z = np.zeros((len(X),len(Y)))
    
    for i in range(len(X)):
        for j in range(len(Y)):
            Z[i,j] = func([X[i],Y[j]])
    
    data = [go.Surface(z=Z)]
    layout = go.Layout(autosize=False,width=500,height=500,margin=dict(l=65,r=50,b=65,t=90))
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='elevations-3d-surface')

## Danger functions

In [38]:
func11 = lambda x: np.sin(x[1]) + np.cos(x[0])
func12 = lambda x: x[0]**4 + x[1]**4
func13 = lambda x: (1-x[0]**2)**2 + (x[0]-x[1])**2
func14 = lambda x: (1-x[0]**3)**2 + (x[1])**4

In [39]:
func11 = lambda x: np.sin(x[1]) + np.cos(x[0])
show_func(func11)

In [41]:
func12 = lambda x: x[0]**4 + x[1]**4
show_func(func12)

In [42]:
func13 = lambda x: 100*(x[1]-x[0]**2)**2 + (1 -x[0])**2
show_func(func13)

In [43]:
func14 = lambda x: (1-x[0]**3)**2 + (x[1])**4
show_func(func14)

## Name of methods

In [46]:
methods = ['golden_ratio','brute_force','half_divide_method','fibonachi','parabol','newtons_method','cubic_interpolation']

#### Comparing algo on current interval [b,a] 

# Function: (x-1)^2

In [47]:
fun_2d(fun1,1,10)

In [48]:
df = pd.DataFrame(columns=['method','x_min','num_iterations','time_duration'])

fun1 = lambda x: (x-1)**2 
#[b,a] = [x0-h,x0+h]
x0 = 1.5
h = 1

for j,method in enumerate(methods):
    
    x_min, n_i, dif_time = test(fun1,x0,h,method, False)

    df.loc[j+1,:] = [method,x_min,n_i,dif_time]
df.transpose()

,1,2,3,4,5,6,7
method,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
x_min,0.999999,0.993691,0.999999,1,1,0.999992,1
num_iterations,26,317,18,26,2,17,1
time_duration,2.81334e-05,0.000191212,3.00407e-05,0.000101805,1.40667e-05,2.6226e-05,0.000120401


In [49]:
compare_bar(df)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [50]:
df_iter = pd.DataFrame(columns=methods)
df_time = pd.DataFrame(columns=methods)

fun1 = lambda x: (x-1)**2 
#[b,a] = [x0-h,x0+h]
x0 = 1.

# h 
steps = [0.00001,0.0001,0.001,0.01,0.025,0.05,0.1,0.25, 0.5,0.75,1,1.25,1.5,1.75,2,2.5,3,5,7,10,15,20,50,100]

for method in methods:
    print(method)
    #print()
    for j,h in enumerate(steps):
        x_min, n_i, dif_time = test(fun1,x0,h,method, False)
        
        df_iter.loc[j,method] = n_i
        df_time.loc[j,method] = dif_time


golden_ratio
brute_force
half_divide_method
fibonachi
parabol
newtons_method
cubic_interpolation


In [51]:
df_iter.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,2,317,1,2,2,17,1
1,7,317,5,6,2,17,1
2,12,317,8,11,2,17,3
3,16,317,11,16,2,17,1
4,18,317,13,18,2,17,1


In [52]:
df_time.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,1.14441e-05,0.000193357,2.69413e-05,2.88486e-05,9.29832e-06,2.07424e-05,0.000124216
1,1.14441e-05,0.00014472,1.26362e-05,1.57356e-05,7.86781e-06,1.88351e-05,9.41753e-05
2,1.43051e-05,0.000143528,1.16825e-05,2.5034e-05,7.86781e-06,1.90735e-05,0.00014925
3,1.43051e-05,0.000148058,1.45435e-05,3.83854e-05,7.86781e-06,2.00272e-05,7.58171e-05
4,1.5974e-05,0.000143051,1.57356e-05,4.43459e-05,8.82149e-06,1.88351e-05,7.55787e-05


In [54]:
show_iters(df_iter)

In [55]:
show_time(df_time)

# Function: sin(x)

In [56]:
fun1 = lambda x: np.sin(x)
x0 = -np.pi /2

In [57]:
fun_2d(fun1,x0,10)

In [58]:
df = pd.DataFrame(columns=['method','x_min','num_iterations','time_duration'])

fun1 = lambda x: np.sin(x)
#[b,a] = [x0-h,x0+h]
x0 = -np.pi /2
h = np.pi /2

for j,method in enumerate(methods):
    
    x_min, n_i, dif_time = test(fun1,x0,h,method, False)

    df.loc[j+1,:] = [method,x_min,n_i,dif_time]
df.transpose()

,1,2,3,4,5,6,7
method,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
x_min,-1.5708,-0.993691,-1.5708,-1.57079,-31873,0,-1.5708
num_iterations,27,317,19,26,28,1,1
time_duration,8.89301e-05,0.000813961,6.46114e-05,0.000144243,0.000114679,1.90735e-05,0.00025773


In [59]:
compare_bar(df)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [60]:
df_iter = pd.DataFrame(columns=methods)
df_time = pd.DataFrame(columns=methods)

fun1 = lambda x: np.sin(x)
#[b,a] = [x0-h,x0+h]
x0 = np.pi /2

# h 
steps = [0.00001,0.0001,0.001,0.01,0.025,0.05,0.1,0.25, 0.5,0.75,1,1.25,1.5,1.75,2,2.5,3,5,7,10,15,20,50,100]

for method in methods:
    #print(method)
    #print()
    for j,h in enumerate(steps):
        x_min, n_i, dif_time = test(fun1,x0,h,method, False)
        
        df_iter.loc[j,method] = n_i
        df_time.loc[j,method] = dif_time

error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0


In [61]:
df_iter.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,2,317,1,2,28,1,0
1,7,317,5,6,28,1,0
2,12,317,8,11,28,1,0
3,16,317,11,16,28,1,0
4,18,317,13,18,28,1,0


In [62]:
df_time.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,2.00272e-05,0.00110817,9.29832e-06,1.95503e-05,9.17912e-05,1.57356e-05,6.91414e-05
1,2.59876e-05,0.000651598,1.78814e-05,2.83718e-05,8.91685e-05,1.07288e-05,4.69685e-05
2,5.60284e-05,0.000576973,2.47955e-05,4.72069e-05,9.05991e-05,1.04904e-05,0.000272036
3,3.98159e-05,0.000564814,3.19481e-05,0.000114918,9.29832e-05,1.07288e-05,0.000114202
4,4.17233e-05,0.000581026,3.67165e-05,0.000128984,8.67844e-05,1.21593e-05,3.69549e-05


In [63]:
show_iters(df_iter)

In [64]:
show_time(df_time)

# Function: exp(x)

In [67]:
fun1 = lambda x: np.exp(x)
x0 = 1

fun_2d(fun1,x0,3)

In [68]:
df = pd.DataFrame(columns=['method','x_min','num_iterations','time_duration'])

fun1 = lambda x: np.exp(x)
x0 = 0
h = 10

for j,method in enumerate(methods):
    print(method)
    x_min, n_i, dif_time = test(fun1,x0,h,method, False)

    df.loc[j+1,:] = [method,x_min,n_i,dif_time]
df.transpose()

golden_ratio
brute_force
half_divide_method
fibonachi
parabol
newtons_method
cubic_interpolation
error 101: grad(x1) must be <= 0


,1,2,3,4,5,6,7
method,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
x_min,-10,-0.993691,-9.99999,-9.99999,-22.3327,-12,NaN
num_iterations,31,317,21,30,100017,13,0
time_duration,0.000166655,0.00127721,6.86646e-05,0.000155926,0.305614,9.08375e-05,3.02792e-05


In [70]:
compare_bar(df)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [71]:
df_iter = pd.DataFrame(columns=methods)
df_time = pd.DataFrame(columns=methods)

fun1 = lambda x: np.sin(x)
#[b,a] = [x0-h,x0+h]
x0 = np.pi /2

# h 
steps = [0.00001,0.0001,0.001,0.01,0.025,0.05,0.1,0.25, 0.5,0.75,1,1.25,1.5,1.75,2,2.5,3,5,7,10,15,20,50,100]

for method in methods:
    #print(method)
    #print()
    for j,h in enumerate(steps):
        x_min, n_i, dif_time = test(fun1,x0,h,method, False)
        
        df_iter.loc[j,method] = n_i
        df_time.loc[j,method] = dif_time

error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0


In [72]:
df_iter.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,2,317,1,2,28,1,0
1,7,317,5,6,28,1,0
2,12,317,8,11,28,1,0
3,16,317,11,16,28,1,0
4,18,317,13,18,28,1,0


In [73]:
df_time.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,1.71661e-05,0.000586271,1.3113e-05,2.57492e-05,0.000239372,1.66893e-05,0.000472546
1,2.19345e-05,0.00056839,3.12328e-05,4.60148e-05,0.000174046,2.09808e-05,0.000221729
2,3.21865e-05,0.000583172,4.29153e-05,7.9155e-05,0.000169992,1.71661e-05,0.000138044
3,6.84261e-05,0.00105786,5.96046e-05,0.000117302,0.000164986,1.52588e-05,0.000122547
4,4.93526e-05,0.00103426,6.50883e-05,0.000134468,0.000156641,1.4782e-05,0.00012517


In [74]:
show_iters(df_iter)

In [75]:
show_time(df_time)

In [76]:
from math import sin, cos, atan, exp

# Function: sin( x^2) + cos( x^2)

In [77]:
fun1 = lambda x: sin(x**2) + cos(x**2)
x0 = 10

fun_2d(fun1,x0,10)

In [78]:
fun1 = lambda x: sin(x**2) + cos(x**2)
x0 = 10
h = 10

df = pd.DataFrame(columns=['method','x_min','num_iterations','time_duration'])

In [441]:
methods = ['golden_ratio','brute_force','parabol','newtons_method','cubic_interpolation']

In [445]:
methods = ['golden_ratio','brute_force','half_divide_method','fibonachi','parabol','newtons_method','cubic_interpolation']

In [79]:
for j,method in enumerate(methods):
    print(method)
    x_min, n_i, dif_time = test(fun1,x0,h,method, False)

    df.loc[j+1,:] = [method,x_min,n_i,dif_time]
df.transpose()

golden_ratio
brute_force
half_divide_method
fibonachi
parabol
newtons_method
cubic_interpolation


,1,2,3,4,5,6,7
method,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
x_min,10.8178,-0.00315457,1.98167,10.8178,1e-05,0,6.92165
num_iterations,31,317,21,30,2,1,28
time_duration,9.44138e-05,0.000505209,0.00014472,0.00022316,1.90735e-05,1.21593e-05,0.00144911


In [80]:
compare_bar(df)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [81]:
df_iter = pd.DataFrame(columns=methods)
df_time = pd.DataFrame(columns=methods)

fun1 = lambda x: sin(x**2) + cos(x**2)
#[b,a] = [x0-h,x0+h]
x0 = 10

# h 
steps = [0.00001,0.0001,0.001,0.01,0.025,0.05,0.1,0.25, 0.5,0.75,1,1.25,1.5,1.75,2,2.5,3,5,7,10,15,20,50,100]

for method in methods:
    #print(method)
    #print()
    for j,h in enumerate(steps):
        x_min, n_i, dif_time = test(fun1,x0,h,method, False)
        
        df_iter.loc[j,method] = n_i
        df_time.loc[j,method] = dif_time

error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 102: grad(x2) must be >= 0
error 102: grad(x2) must be >= 0
error 102: grad(x2) must be >= 0
error 101: grad(x1) must be <= 0
error 102: grad(x2) must be >= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 101: grad(x1) must be <= 0
error 102: grad(x2) must be >= 0
error 102: grad(x2) must be >= 0
error 101: grad(x1) must be <= 0


In [82]:
df_iter.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,2,317,1,2,2,1,0
1,7,317,5,6,2,1,0
2,12,317,8,11,2,1,0
3,16,317,11,16,2,1,0
4,18,317,13,18,2,1,0


In [83]:
df_time.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,1.19209e-05,0.000255585,1.07288e-05,1.28746e-05,1.12057e-05,1.38283e-05,0.000557184
1,2.19345e-05,0.000251293,1.90735e-05,1.81198e-05,9.77516e-06,1.50204e-05,0.00025034
2,1.97887e-05,0.000251055,2.47955e-05,2.98023e-05,9.77516e-06,2.47955e-05,0.000269413
3,3.67165e-05,0.000252724,3.14713e-05,7.03335e-05,9.05991e-06,1.04904e-05,0.000256538
4,2.55108e-05,0.000250101,4.29153e-05,7.67708e-05,9.05991e-06,1.40667e-05,0.000250816


In [84]:
show_iters(df_iter)

In [86]:
show_time(df_time)

# Function: y = |x|

In [87]:
fun1 = lambda x: abs(x)
x0 = 0
h = 10

fun_2d(fun1,x0,h)

In [88]:
df = pd.DataFrame(columns=['method','x_min','num_iterations','time_duration'])



for j,method in enumerate(methods):
    print(method)
    x_min, n_i, dif_time = test(fun1,x0,h,method, False)

    df.loc[j+1,:] = [method,x_min,n_i,dif_time]
df.transpose()


golden_ratio
brute_force
half_divide_method
fibonachi
parabol
newtons_method
cubic_interpolation


,1,2,3,4,5,6,7
method,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
x_min,-2.05303e-06,-0.00315457,-4.76837e-06,-4.3072e-16,1e-05,0,-2.08167e-10
num_iterations,31,317,21,30,2,1,1
time_duration,3.88622e-05,0.00019145,3.91006e-05,0.000202179,1.35899e-05,1.12057e-05,0.000146151


In [89]:
compare_bar(df)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [90]:
df_iter = pd.DataFrame(columns=methods)
df_time = pd.DataFrame(columns=methods)

In [91]:
steps = [0.00001,0.0001,0.001,0.01,0.025,0.05,0.1,0.25, 0.5,0.75,1,1.25,1.5,1.75,2,2.5,3,5,7,10,15,20,50,100]

for method in methods:
    #print(method)
    #print()
    for j,h in enumerate(steps):
        x_min, n_i, dif_time = test(fun1,x0,h,method, False)
        
        df_iter.loc[j,method] = n_i
        df_time.loc[j,method] = dif_time

df_iter.head(5)

,golden_ratio,brute_force,half_divide_method,fibonachi,parabol,newtons_method,cubic_interpolation
0,2,317,1,2,2,1,1
1,7,317,5,6,2,1,1
2,12,317,8,11,2,1,1
3,16,317,11,16,2,1,1
4,18,317,13,18,2,1,1


In [92]:
show_iters(df_iter)

In [93]:
show_time(df_time)